# Die Daten
Die erste Aufgabe soll auch eine kurze Einführung in die Möglichkeiten des Jupyter Notebooks geben. Im Versuch werden wir uns mit Mikroskopbildern beschäftigen. Dies sind die Daten. Wenn man Daten analysieren möchte, ist es immer sinnvoll sich die Daten einmal anzuschauen. Das ist bei Bildern besonders einfach. Zum Anzeigen von Bildern, später auch Plots, werden wir die Python Bibliothek [matplotlib](https://matplotlib.org/) nutzen.
## Ordnerstruktur
Alle für dieses Praktikum nötigen Daten liegen unter /extdata/readonly/f-park-v15. Dies ist eine Pfad-Angabe. Wir arbeiten auf einem Linux-System. Auf Microsoft PCs gehen Pfade immer vom Laufwerk aus, auf dem die Daten liegen (bspw. C:/Users). In Linux ist der toplevel Pfad, auch root-Pfad genannt, /. In Jupyter Notebooks kann man mit einem vorangestellten Ausrufezeichen (!) in einer Zelle einen Kommandozeilenbefehl eingeben. *ls* ist der Kommandozeilenbefehl mit dem man die Inhalte von Ordnern anzeigen kann. Macht euch kurz mit der Ordnerstruktur der Daten vertraut. Um eine Zelle in Jupyter Notebooks auszuführen klickt oben auf **Run** oder verwendet Strg+Enter, mit Umschalt+Enter führt man die Zelle aus und springt automatisch zur nächsten Zelle.

In [ ]:
!ls /extdata/readonly/f-prak-v15

## Grund für die Ordnerstruktur
Im Versuch werden wir zwei verschiedene Datensätze betrachten: *E. Coli* Swarming und *V. Cholerae* Biofilme. Innerhalb der entsprechenden Ordner gibt es zwei Unterordner: train und test. Auf [Wikipedia](https://en.wikipedia.org/wiki/Training,_validation,_and_test_sets) gibt es eine kurze Erklärung für die Unterscheidung von Trainings-, Validierungs- und Testdatensätze. Im train Ordner befinden sich die Daten mit denen trainiert und validiert wird. Im test Ordner sind dementsprechend die Testdaten. Innerhalb eines Unterordners gibt es wieder die Aufteilung in input und labels. Im input Ordner befinden sich die Mikroskopdaten. Diese sollen nun aufbereitet / analysiert werden. Im labels Ordner befinden sich die korrespondieren Bilder, in denen die einzelnen Zellen von Hand bereits segmentiert wurden. Alle Bilder sind im tif-Format abgespeichert.

## Python imports
Wie in vielen anderen Programmiersprachen, muss man auch in Python zuerst angeben, wo die Funktionen definiert sind, die man verwenden möchte. Dies wird über das import-Statement gemacht.
## Jupyter magic und Jupyter ipywidgets
In Jupyter Notebooks gibt es zudem Jupyter Magic. Dies sind Zeilen, die mit einem oder zwei Prozentzeichen (%) beginnen. Damit kann das Verhalten des Notebooks gesteuert werden. Beispiele sind die genaue Darstellung und Interkation mit Plots und Bildern. ipywidgets erlauben eine Interkation mit dem Code über Input Felder, wie bspw. Dropdown Menüs.

Im Folgenden importieren wir die Module, die wir für dieses Notebook benötigen werden und nutzen Jupyter Magic, um die Interaktion mit matplotlib Darstellungen einzustellen.

In [ ]:
# mit einem Doppelkreuz werden in Python Kommentare markiert.
# Kommentarzeilen werden nicht ausgeführt und enthalten keinen Code.

# numpy ist die Standard Library in Python, um numerische Operationen mit Arrays durchzuführen.
import numpy as np
import ipywidgets as widgets
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from glob import glob
from tifffile import imread

from stardist import random_label_cmap
from csbdeep.utils import Path
lbl_cmap = random_label_cmap()

In [ ]:
# Wir lesen alle Trainingsdaten ein.
# Im folgenden Code steht X stets für input und Y für labels.
# Mit glob suchen wir einfach nur nach Pfaden, die dem Suchmuster entsprechen.
X_glob = sorted(glob('/extdata/readonly/f-prak-v15/e-coli-swarming/train/input/*.tif'))
Y_glob = sorted(glob('/extdata/readonly/f-prak-v15/e-coli-swarming/train/labels/*.tif'))
def labelname(name):
    return name[:-5]+name[-4:]
# Wir überprüfen, ob für jede Datei im input auch ein entsprechendes label existiert.
# Falls dies nicht der Fall ist, wird hier ein Fehler erzeugt.
assert all(Path(x).name == labelname(Path(y).name) for x,y in zip(X_glob, Y_glob))

In [ ]:
# Nun wollen wir auch die Bilder in den entsprechenden Pfaden öffnen.
X = list(map(imread, X_glob))
Y = list(map(imread, Y_glob))
# X und Y sind Listen. Die Elemente der Listen sind die Bilder.
imgs_dropdown = widgets.Dropdown(
    options=list(range(len(X))),
    value=0,
    description='Welches Bild wollen wir betrachten?',
    style={'description_width': 'initial'}
    )
display(imgs_dropdown)

In [ ]:
i = imgs_dropdown.value
# Wir speichern in img und lbl den input und das label des ausgewählten Bildes.
img, lbl = X[i], Y[i]

## Bilder als numpy Arrays / Tensoren
Bilder werden als numpy-Arrays gespeichert. Meist haben diese Arrays die Dimension 2 oder 3. Dies entspricht den zwei räumlichen Achsen x und y, sowie einer möglichen Dimension für die Farbkanäle. **Wichtig:** Im Gegensatz zur Physik oder Mathematik erfolgt die räumliche Indexierung andersherum. In der Physik schreiben wir für eine Funktion meist f(x, y). In Python greifen wir auf das Pixel eines Bildes via img\[index_y, index_x\] zu. Manchmal haben auch Graustufenbilder eine dritte Dimension. Im folgenden lassen wir uns die Dimension und den shape der Bilder anzeigen.

In [ ]:
print("Dimension von img:\t", img.ndim)
print("Shape von img:\t\t", img.shape)
print("Dimension von lbl:\t", lbl.ndim)
print("Shape von lbl:\t\t", lbl.shape)

Im obigen Code musste viel wiederholt werden und doppelt geschrieben werden. Folgender Code erzeugt die gleiche Ausgabe und ist leichter zu warten:

In [ ]:
for name, arr in zip(["img", "lbl"], [img, lbl]):
    print("Dimension von {}:\t".format(name), arr.ndim)
    print("Shape von {}:\t\t".format(name), arr.shape)

In [ ]:
# Wir erzeugen eine Figure
plt.figure(figsize=(16,10))
# In einer Figure kann es verschiedene subplots geben. Mit 121 wählen wir einen dieser Plots aus.
# die erste Zahl (1) gibt die Anzahl der Zeilen der Figure an.
# Die zweite Zahl (2) ist die Anzhal der Spalten
# Mit der dritten Zahl (1) wählen wir den ersten der Subplots aus.
plt.subplot(121)
# dies ist der Befehl um das Bild darzustellen
plt.imshow(img, cmap='gray')
#
# Was macht dieser Befehl ???
#
plt.axis('off')
# Hiermit wird dem Subplot ein Titel gegeben
plt.title('Raw image')
plt.subplot(122)
plt.imshow(lbl, cmap='gray')
#plt.imshow(lbl, cmap=lbl_cmap)
plt.axis('off')
plt.title('GT labels')
None;

## Colormaps
Graustufenbilder bestehen aus Intensitätswerten, die meist nicht einer physikalischen Größe zugeordnet werden können und daher mit der Einheit a.u. versehen werden. Dies bedeutet auch, dass sie keine "echte" Farbe besitzen. Das obige Bild wurde als Graustufenbild dargestellt: große Intensitäten werden weiß, kleine Intensitäten schwarz dargestellt. Könnt ihr erklären, wie die Darstellung des linken Bildes zustande kommt?

Mit Colormaps kann die Darstellung von Bildern verändert werden. Dies bedeutet nicht, dass die Intensitäten des zugrundeliegenden Bildes verändert werden. Spielt ein bisschen mit den Colormaps rum. Wenn ihr keine cmap im Befehl imshow angebt, wird die Standard Colormap verwendet. Es gibt auch die Colormaps jet und viridis. Was macht die Colormap lbl_cmap?

## Scalebar
In der Physik ist die Achsenbeschriftung von Graphen wichtig. Ebenso ist in der Biologie wichtig, dass man in Bildern angibt, wie groß die abgebildeten Strukturen sind. Dies wird mit einer Scalebar gemacht. Dafür gibt es das Modul [matplotlib-scalebar](https://pypi.org/project/matplotlib-scalebar/). Modifiziert die Figure, so dass eine Scalebar angezeigt wird. Die Pixel Size beträgt µm/px.

**Randbemerkung:** Was ist der Unterschied zwischen Pixel Size und Auflösung eines Bildes?

In [ ]:
from matplotlib_scalebar.scalebar import ScaleBar

scalebar = ScaleBar(0.2) # 1 pixel = 0.2 meter
plt.gca().add_artist(scalebar)

## Aufgabe 1-1
Betrachtet die obige Figure und überlegt, um was es in diesem Versuch geht: Wir wollen Algorithmen entwerfen, mit denen wir obige Segmentierung automatisieren können. Um zu überprüfen, ob die automatische Segmentierung des Computers gut ist, ist es sinnvoll, die Segmentierung mit der Segmentierung zu vergleichen. Im obigen Vergleich ist es jedoch schwer zu erkennen, ob Segmentierung und Input im Detail zusammenpassen. Das wäre einfacher zu erzielen, wenn die Segmentierung über den Input gelegt werden würde. Erstellt eine Figure, in der genau das gemacht wird. Ergänzt zudem eine Möglichkeit nur einen Ausschnit der Bilder darzustellen.
### Hinweise (nur falls benötigt)
- Durch mehrmaliges Ausführen von imshow, können mehrere Bilder in den gleichen subplot geplottet werden
- mit dem Argument alpha kann man die Deckkraft eines Bildes verringern
- mit slices und start:stop kann man Array Ausschnitte indizieren